In [ ]:
import graphlab

In [ ]:
book_ratings_data = graphlab.SFrame.read_csv("BX-Book-Ratings.csv", delimiter=";")

In [ ]:
book_ratings_data.head()

In [ ]:
high_rated_data = book_ratings_data[book_ratings_data["Book-Rating"] >= 8]
low_rated_data = book_ratings_data[book_ratings_data["Book-Rating"] < 8]

train_data_1, test_data = graphlab.recommender.util.random_split_by_user(high_rated_data, 
                                                                         user_id="User-ID", item_id="ISBN")
train_data = train_data_1.append(low_rated_data)

In [ ]:
def my_popularity_recommender(train_data=None, user_id="user_id", item_id="item_id", 
                              user_data=None, item_data=None, rating="rating"):
    rating_10 = train_data[train_data[rating] == 10]
    popular_books = rating_10.groupby(key_columns=item_id, 
                                              operations={"count": graphlab.aggregate.COUNT()})
    popular_books = popular_books.sort("count", ascending=False)
    
    print popular_books
    pos_list = []
    for i in range(len(item_data[item_id])):
        if len(pos_list) == 5:
            break
        elif popular_books[item_id][i] in item_data[item_id]:
            pos_list.append(i)
    
    most_popular_books_ids = []
    most_popular_books = []
    for pos in pos_list:
        most_popular_books_ids.append(popular_books[item_id][pos])
        
    for ids in most_popular_books_ids:
        most_popular_books.append(item_data[item_data[item_id] == ids][["Book-Title", "Book-Author", 
                                                                          "Year-Of-Publication", "Publisher"]][0])
    return most_popular_books, most_popular_books_ids

In [ ]:
data_book = graphlab.SFrame.read_csv("BX-Books.csv")

In [ ]:
data_book = data_book[["ISBN", "Book-Title", "Book-Author", "Year-Of-Publication", "Publisher"]]

In [ ]:
recommended_book, book_id = my_popularity_recommender(train_data, item_data=data_book,
                                             user_id="User-ID", item_id="ISBN", rating="Book-Rating")

In [ ]:
book_id

In [ ]:
len(test_data["User-ID"].unique())

In [ ]:
unique_user = train_data["User-ID"].unique()
shown_books = 5
liked_list = []
liked_and_shown_list = []
for user in unique_user:
    count = 0
    rated_books = test_data[test_data["User-ID"] == user]["ISBN"]
    liked_list.append(len(rated_books))
    for book in rated_books:
        if book in book_id:
            count += 1
    liked_and_shown_list.append(count)